In [20]:
import re
import cv2
import tensorflow as tf
import numpy as np

In [55]:
CAMERA_WIDTH = 320
CAMERA_HEIGHT = 320

def load_labels(path='labels.txt'):
    """Loads the labels file. Supports files with or without index numbers."""
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        labels = {}
        for row_number, content in enumerate(lines):
            pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
            # print(pair)
            if len(pair) == 2 and pair[0].strip().isdigit():
                labels[int(pair[0])] = pair[1].strip()
            else:
                labels[row_number] = pair[0].strip()
    return labels

def set_input_tensor(interpreter, image):
    """Sets the input tensor."""
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = np.expand_dims((image-255)/255, axis=0)


def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor


def detect_objects(interpreter, image, threshold):
    """Returns a list of detection results, each a dictionary of object info."""
    set_input_tensor(interpreter, image)
    interpreter.invoke()
    # Get all output details [harusnya ini yang bener; boxes, classes, scores, dan count]
    # boxes = get_output_tensor(interpreter, 0)
    # classes = get_output_tensor(interpreter, 1)
    # scores = get_output_tensor(interpreter, 2)
    # count = get_output_tensor(interpreter, 3)
    
    scores = get_output_tensor(interpreter, 0)
    boxes = get_output_tensor(interpreter, 1)
    count = int(get_output_tensor(interpreter, 2))
    classes = get_output_tensor(interpreter, 3)
    
    results = []
    for i in range(count):
        if scores[i] >= threshold:
            result = {
                'bounding_box': boxes[i],
                'class_id': classes[i],
                'score': scores[i]
            }
            results.append(result)
    return results

In [56]:
labels = load_labels()
MODEL_PATH = r'.\Tensorflow\workspace\models\my_ssd_mobnet\tfliteexport\saved_model\detect.tflite'
interpreter = tf.lite.Interpreter(MODEL_PATH)
interpreter.allocate_tensors()
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

In [57]:
labels

{0: 'botolplastik',
 1: 'botolkaca',
 2: 'plastik',
 3: 'kertas',
 4: 'kardus',
 5: 'sedotan',
 6: 'sterofoam'}

In [58]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    img = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (320,320))
    res = detect_objects(interpreter, img, 0.8)
    print(res)

    for result in res:
        ymin, xmin, ymax, xmax = result['bounding_box']
        xmin = int(max(1,xmin * CAMERA_WIDTH))
        xmax = int(min(CAMERA_WIDTH, xmax * CAMERA_WIDTH))
        ymin = int(max(1, ymin * CAMERA_HEIGHT))
        ymax = int(min(CAMERA_HEIGHT, ymax * CAMERA_HEIGHT))
        
        cv2.rectangle(frame,(xmin, ymin),(xmax, ymax),(0,255,0),3)
        cv2.putText(frame,labels[int(result['class_id'])],(xmin, min(ymax, CAMERA_HEIGHT-20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),2,cv2.LINE_AA) 

    cv2.imshow('Pi Feed', frame)

    if cv2.waitKey(10) & 0xFF ==ord('q'):
        cap.release()
        cv2.destroyAllWindows()

[{'bounding_box': array([0.14485851, 0.37404978, 0.9670702 , 0.7439277 ], dtype=float32), 'class_id': 6.0, 'score': 0.99115163}]
[{'bounding_box': array([0.14640242, 0.37286606, 0.970059  , 0.7466227 ], dtype=float32), 'class_id': 6.0, 'score': 0.9920494}]
[{'bounding_box': array([0.14244005, 0.37494594, 0.9717376 , 0.7432224 ], dtype=float32), 'class_id': 6.0, 'score': 0.99252653}]
[{'bounding_box': array([0.14697468, 0.37546462, 0.9692575 , 0.7390284 ], dtype=float32), 'class_id': 6.0, 'score': 0.99052054}]
[{'bounding_box': array([0.14697468, 0.37546462, 0.9692575 , 0.7390284 ], dtype=float32), 'class_id': 6.0, 'score': 0.99052054}]
[{'bounding_box': array([0.14702573, 0.37489086, 0.96406114, 0.7450375 ], dtype=float32), 'class_id': 6.0, 'score': 0.9920468}]
[{'bounding_box': array([0.14702573, 0.37489086, 0.96406114, 0.7450375 ], dtype=float32), 'class_id': 6.0, 'score': 0.9920468}]
[{'bounding_box': array([0.14750174, 0.37411678, 0.97232604, 0.75443876], dtype=float32), 'class_id'

In [42]:
cap.release()
cv2.destroyAllWindows()